In [ ]:
#  example=   {"adult": 1.00,
#       "senior": 0.50,
#             "student": 0.50,
#                   "medicare_medicaid": 0.50,
#                         "veterans_active_military": 0.50,
#                                 "pass": {"duration_hours": 24, "cost": 3.00},
#                                         "school_free_fare": True,  # Northern Illinois University (NIU) students ride free with valid NIU OneCard
#                                                 "live_fare_link": "https://www.cityofdekalb.com/430/Bus-Routes-and-Schedules"}

In [ ]:
import json
import os
from openai import OpenAI
from pprint import pprint

# Initialize the OpenAI client
api_key = 'whoknowsthekey?'
client = OpenAI(api_key=api_key)

# File path for saving the fare data
output_file = 'fare_data.json'

# Function to load existing fare data if the JSON file exists
def load_existing_data(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return json.load(f)
    return {}

# Function to call OpenAI API and fetch fare data for each agency
def fetch_fare_data(dba):
    # dba = "City of Cedar Rapids, dba: Cedar Rapids Transit"
    prompt = f"""
    Please format your response as a python dictionary.

    Give me the {dba}'s current live link to its Fares about its bus service. 
    Do not use the DoT's data, use the {dba}'s own website. 
    For example if the adult fare is $0.50 then make "adult": 0.50. If the senior fare is free then do "senior": 0.00, if it is 0.25 then do "senior": 0.25.
    If there is a special extended duration pass, do "pass": {{"duration_hours": 24, "cost": 3.00}} for example.
    if there isnt, do "pass": None
    If there is "medicare_medicaid", include it. If there is "veterans_active_military", include it. 
    If there is a special collaboration with a university that gives free fare, say "school_free_fare": True, else make it False.
    If there is a "student" cost, include it. Also student pass, "student_pass": {{"duration": "semester", "cost": 60.00}} for example.
    If there is a link to the live fares, include it as "live_fare_link": "mylinkhere"
    If there is a rate depending on age that notes specific years, do "age_discount": {{"12-18": 0.50, "65+": 0.50}}, 
    or if it is a child boarding with an adult, do "age_discount": {{"with_adult": 0.00}}, or "age_discount": {{"may_qualify": True}} if special terms.

    Lastly, give me any notes i may want to know, for example, fares suspended due to COVID, with "notes": "your cool notes here"
    DO NOT INCLUDE FARES RELATING TO PARATRANSIT. BUS ONLY.
    """
 
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="chatgpt-4o-latest",
    )
    # print(chat_completion)
    response = eval(chat_completion.choices[0].message.content)  # Correct access
    pprint(response)
    return response

# Function to save fare data to a JSON file
def save_fare_data(file_path, fare_data):
    with open(file_path, 'w') as f:
        json.dump(fare_data, f, indent=4)

# Function to append fare data for a given number of rows
def append_fare_data(df, num_rows):
    # Load existing fare data
    fare_data = load_existing_data(output_file)
    
    # Find where we left off
    processed_agencies = set(fare_data.keys())
    new_rows = df[~df['Agency'].isin(processed_agencies)].head(num_rows)

    # Process each new row
    for index, row in new_rows.iterrows():
        dba = row['Agency']
        
        # Fetch fare data from API
        fare_info = fetch_fare_data(dba)
        
        # Add the fare data to the dictionary
        fare_data[dba] = fare_info

        # Save the updated fare data to the JSON file
        save_fare_data(output_file, fare_data)

    print(f"Processed {len(new_rows)} new rows.")

# Example usage: proceed with the next 6 rows
# append_fare_data(sorted_df, 4)


In [ ]:
import json
import os
import tkinter as tk
from tkinter import messagebox
import pyperclip

# File path for saving the fare data
output_file = 'fare_data.json'

# Function to load existing fare data if the JSON file exists
def load_existing_data(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return json.load(f)
    return {}

# Function to save fare data to a JSON file
def save_fare_data(file_path, fare_data):
    with open(file_path, 'w') as f:
        json.dump(fare_data, f, indent=4)

# Function to handle the Copy button click
def copy_prompt(dba):
    prompt = f"""
    Please format your response as a python dictionary.

    Give me the {dba}'s current live link to its Fares about its bus service. 
    Do not use the DoT's data, use the {dba}'s own website. 
    For example if the adult fare is $0.50 then make "adult": 0.50. If the senior fare is free then do "senior": 0.00, if it is 0.25 then do "senior": 0.25.
    However, if the pass is a range, such as it costs money depending on how much zones you pass through,
    provide a range of costs, if you are able to.
    If there is a special extended duration pass, do "pass": {{"duration_hours": 24, "cost": 3.00}} for example.
    if there isnt, do "pass": None
    If there is "medicare_medicaid", include it. If there is "veterans_active_military", include it. 
    If there is a special collaboration with a university that gives free fare, say "school_free_fare": True, else make it False.
    If there is a "student" cost, include it. Also student pass, "student_pass": {{"duration": "semester", "cost": 60.00}} for example.
    If there is a link to the live fares, include it as "live_fare_link": "mylinkhere"
    If there is a rate depending on age that notes specific years, do "age_discount": {{"12-18": 0.50, "65+": 0.50}}, 
    or if it is a child boarding with an adult, do "age_discount": {{"with_adult": 0.00}}, or "age_discount": {{"may_qualify": True}} if special terms.

    Lastly, give me any notes i may want to know, for example, fares suspended due to COVID, with "notes": "your cool notes here"
    and give the copy-pasted data in "data": "your data here" so i can confirm.
    You are supposed to rip it straight from the webpage for the data field. Copy and paste what you found.
    DO NOT FORGET THE DATA FIELD.
    DO NOT INCLUDE FARES RELATING TO PARATRANSIT. BUS ONLY.
    """
    pyperclip.copy(prompt)
    # messagebox.showinfo("Copied", "Prompt copied to clipboard!")

# Function to handle the Save button click
def save_data(dba, text_box, counter_label, current_index, total, window):
    data = text_box.get("1.0", tk.END).strip()
    
    try:
        fare_info = eval(data)  # Ensure the data is a valid dictionary
        fare_data = load_existing_data(output_file)
        fare_data[dba] = fare_info
        save_fare_data(output_file, fare_data)
        
        # Update the counter label
        counter_label.config(text=f"{current_index + 1} / {total} to go")
        
        # Close the current window to proceed to the next agency
        window.destroy()
        
    except Exception as e:
        messagebox.showerror("Error", f"Failed to save data: {e}")

# Function to create the GUI for each DBA
def show_dba_window(dba, current_index, total):
    window = tk.Tk()
    window.title(f"Fare Data for {dba}")

    # Display the DBA label
    label = tk.Label(window, text=f"Current DBA: {dba}")
    label.pack(pady=10)

    # Copy button
    copy_button = tk.Button(window, text="Copy Prompt", command=lambda: copy_prompt(dba))
    copy_button.pack(pady=5)

    # Text box for pasting and saving the dictionary
    text_box = tk.Text(window, height=15, width=70)
    text_box.pack(pady=10)

    # Counter label showing the current DBA progress
    counter_label = tk.Label(window, text=f"{current_index + 1} / {total} to go")
    counter_label.pack(pady=5)

    # Save button
    save_button = tk.Button(window, text="Save Data", command=lambda: save_data(dba, text_box, counter_label, current_index, total, window))
    save_button.pack(pady=5)

    # Bind Enter key to replicate Save button click
    window.bind('<Return>', lambda event: save_data(dba, text_box, counter_label, current_index, total, window))

    window.mainloop()

# Function to iterate over the DataFrame and show a window for each DBA
def process_dbas(df):
    # Load existing fare data
    fare_data = load_existing_data(output_file)
    
    # Filter the rows to process
    new_rows = df[~df['Agency'].isin(fare_data.keys())]
    
    total_rows = len(new_rows)

    # Loop through each unprocessed DBA row
    for current_index, (index, row) in enumerate(new_rows.iterrows()):
        dba = row['Agency']
        show_dba_window(dba, current_index, total_rows)


# Example usage: start processing all DBAs
# process_dbas(df)


In [ ]:
import pandas as pd

# Assuming df is already defined
# Save the DataFrame to a pickle file
df.to_pickle('ntd.pkl')

